In [ ]:
import holoviews as hv; hv.extension('bokeh', logo=None)
import panel as pn;     pn.extension()
hv.opts.defaults(hv.opts.Raster(cmap='gray', xaxis=None, yaxis=None, frame_width=200, aspect='equal'))

import numpy as np
import os

In [ ]:
# Define randomized SVD function
def rSVD(X,r,q,p):
    # Step 1: Sample column space of X with P matrix
    ny = X.shape[1]
    P = np.random.randn(ny,r+p)
    Z = X @ P
    for k in range(q):
        Z = X @ (X.T @ Z)

    Q, R = np.linalg.qr(Z, mode='reduced')

    # Step 2: Compute SVD on projected Y = Q.T @ X
    Y = Q.T @ X
    UY, S, VT = np.linalg.svd(Y, full_matrices=0)
    U = Q @ UY

    return U, S, VT

In [ ]:
from PIL import Image, ImageOps
A = np.array(Image.open( os.path.join('..','DATA','jupiter.jpg') ))

X = np.mean(A,axis=2) # Convert RGB -> grayscale

U, S, VT = np.linalg.svd(X,full_matrices=0) # Deterministic SVD

r = 400 # Target rank
q = 1   # Power iterations
p = 5   # Oversampling parameter

rU, rS, rVT = rSVD(X,r,q,p)

In [ ]:
## Reconstruction
XSVD   = U[:,:(r+1)] @ np.diag(S[:(r+1)]) @ VT[:(r+1),:] # SVD approximation
errSVD = np.linalg.norm(X-XSVD,ord=2) / np.linalg.norm(X,ord=2)

XrSVD  = rU[:,:(r+1)] @ np.diag(rS[:(r+1)]) @ rVT[:(r+1),:] # SVD approximation
errSVD = np.linalg.norm(X-XrSVD,ord=2) / np.linalg.norm(X,ord=2)

In [ ]:
(hv.Raster(X)+hv.Raster(XSVD)+hv.Raster(XrSVD)).opts(shared_axes=False)

In [ ]:
## Illustrate power iterations
X        = np.random.randn(1000,100)
U, S, VT = np.linalg.svd(X,full_matrices=0)
S        = np.arange(1,0,-0.01)
X        = U @ np.diag(S) @ VT

sigmas = [hv.Curve(X, "index", "sigma", label='SVD')]
Y = X
for q in range(1,6):
    Y = X.T @ Y
    Y = X @ Y
    Uq, Sq, VTq = np.linalg.svd(Y,full_matrices=0)
    sigmas.append( hv.Curve( Sq , label=f"rSVD: q={q}"))

hv.Overlay(sigmas).opts( hv.opts.Curve( width=600 )).opts(title='Effect of Power Iteration')